# Process Names: Get Author names from the dataset

In this notebook we create three functions: `load_data`, `get_author_names_list`, and `extract_names`.

## Loading the Data

In [ ]:
#| default_exp process_names

In [ ]:
#| export

import pandas as pd
import pprint
import re

In [ ]:
#| export

#Reading Excel file with pandas and choosing the sheet we want to work with
usecols = ['id', 'title', 'contact_email', 'contact_author_name', 'doi', 'author_names']

df = pd.read_excel(open('Catalogdatabase-till2018b.xlsx', 'rb'), sheet_name='publication', usecols=usecols)

# Practice functions with a small subset of the entire df
df_small = df.head()

#Displaying Relevant fields we'll work with

df_small

That looks really good, but can we turn that into a function so we don't have to call that every time we want to use the dataset in the notebook?

In [ ]:
#| export
# TODO: write a function to load the data. Create a parameter called `small` that let's us chose whether to
# return the whole dataframe or just the head

def load_data(small=False):
    return None # return the dataframe

In [ ]:
# get the first row
single_publication = df_small.loc[0]
single_publication

In [ ]:
# get the author list from the first row
author_names = single_publication['author_names']
author_names

In [ ]:
# remove all the brackets and single quotes
_author_names = author_names.strip("[]").replace("'", "")
_author_names

In [ ]:
# Split at ',' to get a list
_author_names = _author_names.split(', ')
_author_names

In [ ]:
#| export
def get_author_names_list(author_names):
    author_names = author_names.strip("[]").replace("'", "")
    author_names_list = author_names.split(', ')
    return author_names_list

In [ ]:
author_names = single_publication['author_names']
author_names_list = get_author_names_list(author_names)
author_names_list

In [ ]:
# grab a single author name from the list of authors
single_author = author_names_list[3]
single_author

In [ ]:
## Extract the first, middle, and last name
names = single_author.split(' ')
first_name = names[0]
last_name = names[-1]
middle_name = ' '.join(names[1:-1]) if len(names) > 2 else None

(last_name, first_name, middle_name)

That looks good! Let's put all that logic in a function we can reuse.

In [ ]:
def extract_names(full_name):
    names = full_name.split(' ')
    first_name = names[0]
    last_name = names[-1]
    middle_name = ' '.join(names[1:-1]) if len(names) > 2 else None
    return (last_name, first_name, middle_name)

In [ ]:
extract_names(single_author)

Now let's try that function on an author name that has a different format.

In [ ]:
first_author_in_fourth_pub = get_author_names_list(df_small.loc[4]['author_names'])[0]
first_author_in_fourth_pub

In [ ]:
extract_names(first_author_in_fourth_pub)

That didn't really do it. Let's try again.

In [ ]:
def extract_names(full_name):
    names = full_name.split()
    first_name = names[0][0]
    middle_name = names[0][1:] if len(names[0]) > 1 else None
    last_name = names[-1]
    return (last_name, first_name, middle_name)

In [ ]:
extract_names(first_author_in_fourth_pub)

Much better! What about Ang Yang? 

In [ ]:
extract_names(single_author)

Not exactly. Try writing a function that works for both cases. Let's write some test cases to check against every time we iterate on the function we come up with. We will do this with `assert` statements, which are this simplest way to do unit testing. The code below will throw an error if the code after the `assert` keyword returns False. Let's try the 'SP Brown' example first since we know it's working. 

In [ ]:
assert extract_names('SP Brown') == ('Brown', 'S', 'P'), "The extract_names function isn't working as expected. Run extract_names('SP Brown') in it's own cell to see what happened!"

Okay great it ran as expected! Now let's try an assert statement on something that doesn't run as expected. It will throw an error with a message that matches the string to the right of the assert statement (just after the last comma).

In [ ]:
assert extract_names('Ang Yang') == ('Yang', 'Ang', None), "The extract_names function isn't working as expected. Run extract_names('Ang Yang') in it's own cell to see what happened!"

Okay! Now we know how to write assert stamements to check our code ever time we make a small change! That's awesome because it means we don't have to go through all those cells before to change our workflow a little and try another name. Instead, we can just make some up!

In [ ]:

# TODO: make sure this function works for the SP Brown and Ang Yang cases
def extract_names(full_name):
    # Check for any name with first two capital letters
    pattern_first_two_capital = re.compile(r'^([A-Z])([A-Z])\s+(.*)$')
    match_first_two_capital = pattern_first_two_capital.match(full_name)

    if match_first_two_capital:
        first_name = match_first_two_capital.group(1)
        middle_name = match_first_two_capital.group(2)
        last_name = match_first_two_capital.group(3)
    else:
        # Fallback to the original splitting
        names = full_name.split(' ')
        first_name = names[0]
        middle_name = ' '.join(names[1:-1]) if len(names) > 2 else None
        last_name = names[-1]

    return (last_name, first_name, middle_name) # note that I changed the order here

In [ ]:
assert extract_names('SP Brown') == ('Brown', 'S', 'P')
assert extract_names('Ang Yang') == ('Yang', 'Ang', None)
assert extract_names('Ang F Yang') == ('Yang', 'Ang', 'F')

That looks great! But it still doesn't account for a lot of the funny edge cases we need to expect. Let's make sure that multiple middle names or initials are accounted for. Let's expand on the `extract_names` function you wrote above so that it accounts for more cases. Just below this function we will have some assert statements so you can check your work.

**Note**: for this version we will need to return more middle names. Let's assume that there are never more than three middle names. 

In [ ]:
def extract_names(full_name):
    pass # TODO: copy-paste your function from above and modify it so that it accounts for new edge cases

In [ ]:
assert extract_names('SP Brown') == ('Brown', 'S', 'P', None, None)
assert extract_names('Ang Yang') == ('Yang', 'Ang', None, None, None)
assert extract_names('RBBF Burnett') == ('Burnett', 'R', 'B', 'B', 'F')
assert extract_names('Rebecca BB Burnett') == ('Burnett', 'Rebecca', 'B', 'B', None)

In [ ]:
# TODO: create some of your own assert statements to check funny edge cases

Nice! Now let's use that return value to create a new Author object.

## Review

Let's put everything we did all together.

In [ ]:
single_publication = df.loc[0]
single_publication

In [ ]:
author_names = single_publication['author_names']
author_names

In [ ]:
author_names_list = get_author_names_list(author_names)
author_names_list

In [ ]:
first_author = author_names_list[0]

In [ ]:
last, first, middle, middle2, middle3 = extract_names(first_author)
(last, first, middle)

In [ ]:
# TODO: when you are all finished...
# 1. Restart kernel and clear all outputs
# 2. save this notebook
# 3. Run this cell

from nbdev.export import nb_export
nb_export('process_names.ipynb', 'preprocessing')